In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('train_kaggle.csv')
test_data = pd.read_csv('test_kaggle.csv')

In [172]:
df.head()

,Store_id,SKU_id,Date,Promo,Demand,Regular_Price,Promo_Price
0,1,1,01.01.2015,NaN,22,163.78,NaN
1,1,1,02.01.2015,NaN,41,163.78,NaN
2,1,1,03.01.2015,NaN,35,163.78,NaN
3,1,1,04.01.2015,NaN,72,163.78,NaN
4,1,1,05.01.2015,NaN,25,163.78,NaN


In [173]:
len(df)

86916

In [174]:
# Fill promo with 0 and extract temporal features
df['Promo'] = df['Promo'].fillna(0.0)
df['Date'] = pd.to_datetime(df['Date'], dayfirst = True)
df["weekday"] = df.Date.dt.weekday
df["monthday"] = df.Date.dt.day
df['is_weekend'] = df.weekday.isin([5,6])*1
df['month'] = df.Date.dt.month

In [175]:
df['Promo_Price'] = df['Promo_Price'].fillna(0)

In [176]:
sns.mpl.rc("figure", figsize=(25, 5))
sns.mpl.rc("font", size=14)

In [177]:
# as can be seen - months 1 and 2 are lower than usual, months 7 and 11 - higher
# use it as a feature

ys = df.groupby('month').Demand.mean()
ys

month
1     114.524155
2     176.239268
3     209.662368
4     199.593624
5     186.661030
6     197.552030
7     317.278839
8     213.674271
9     184.899961
10    203.475361
11    247.375093
12    146.560180
Name: Demand, dtype: float64

In [178]:
df['expected_lower'] = [int(x in (1,2)) for x in df.month]
df['expected_higher'] = [int(x ==11 or x ==7) for x in df.month]
for j in range(len(df)):
    if(df['Promo_Price'][j] == 0):
        df['Promo_Price'][j] =df['Regular_Price'][j] 

/var/folders/kt/8pnx4rb10372fz63y2z655nh0000gn/T/ipykernel_31657/873926764.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Promo_Price'][j] =df['Regular_Price'][j]


In [179]:
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split as tts
target = df.Demand
df.drop('Demand',axis=1,inplace=True)

In [180]:
X_train, X_test, y_train, y_test = tts(
   df, target, test_size=0.2, random_state=42)

In [181]:
numerical_cols = ['Regular_Price','weekday','monthday','Promo_Price','month']
scal = StandardScaler()
X_train[numerical_cols] = scal.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = scal.transform(X_test[numerical_cols])

/Users/michaelmiller/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
/Users/michaelmiller/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [182]:
from xgboost import XGBRegressor
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error as mae

In [183]:
X_train.drop('Date',axis=1,inplace=True)
X_test.drop('Date',axis=1,inplace=True)
X_train.drop('Store_id',axis=1,inplace=True)
X_test.drop('Store_id',axis=1,inplace=True)
X_train.drop('SKU_id',axis=1,inplace=True)
X_test.drop('SKU_id',axis=1,inplace=True)

/Users/michaelmiller/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [184]:
model = XGBRegressor(n_estimators=100, max_depth=10)
model.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             objective='reg:squarederror', predictor=None, ...)

In [185]:
y_preds = model.predict(X_test)

In [186]:
mae(y_test,y_preds)

98.63285298635967

In [113]:
y_preds

array([ 2.8095917e+02,  3.5682044e+00,  1.8670254e+00, ...,
        5.1549634e+02,  7.8382607e+00, -8.6920317e-03], dtype=float32)

In [169]:
X_train

,Store_id,SKU_id,Date,Promo,Demand,Regular_Price,Promo_Price,weekday,monthday,is_weekend,month,expected_lower,expected_higher
38327,78,1,2015-06-14,1.0,924,-0.817720,-1.373594,1.491958,-0.176079,1,0.125273,0,0
24627,49,1,2015-09-01,0.0,145,-0.166671,0.122405,-1.008821,-1.666127,0,0.991008,0,0
68925,50,2,2015-10-28,0.0,0,0.513773,0.631942,-0.508665,1.428587,0,1.279586,0,0
84058,84,2,2015-11-30,0.0,6,-0.092639,0.177843,-1.508977,1.657825,0,1.568165,0,1
50839,14,2,2016-05-17,0.0,0,-0.077397,0.189256,-1.008821,0.167778,0,-0.163306,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6265,13,1,2015-06-19,0.0,245,-1.202035,-0.652906,0.491646,0.397016,0,0.125273,0,0
54886,22,2,2016-04-30,0.0,0,-0.077397,0.189256,0.991802,1.657825,1,-0.451884,0,0
76820,66,2,2016-04-11,0.0,3,-0.077397,0.189256,-1.508977,-0.519936,0,-0.451884,0,0
860,2,1,2015-12-19,0.0,443,-0.965785,-0.475995,0.991802,0.397016,1,1.856743,0,0
